# Importar Modulos

In [3]:
import pandas as pd
import numpy as np
import seaborn as sn
import plotly.express as px
import tensorflow as tf

from matplotlib import pyplot as plt
from sklearn import preprocessing
from IPython.display import display

# Importación de Datos

In [4]:
data = np.loadtxt("lp5.csv", delimiter=",", dtype=str)
data_copy = data #va de 0 a 2623 o sea 2624 datos
cont = 16
labels = []
numbers = []
features= np.zeros((164,15,6))

for i in range(len(data_copy)):
    if(cont == 16):
        labels.append(data_copy[i])
    if(cont<16):
        numbers.append(data_copy[i])
    cont -= 1
    if(cont == 0):
        cont = 16
        
cont = 0
for i in range(164):
    for j in range(15):
        for z in range(6):
            features[i][j][z] = numbers[cont][z]
        cont += 1
        
labels_df = pd.DataFrame(labels, columns=['labels','1','2','3','4','5'])
labels_df = labels_df.loc[:,['labels']]
display(labels_df)
display(features)

,labels
0,normal
1,normal
2,normal
3,normal
4,normal
...,...
159,collision_in_tool
160,collision_in_tool
161,collision_in_tool
162,collision_in_tool


array([[[ -2.,  -1.,  81.,   0.,  -5.,   0.],
        [ -2.,  -1.,  79.,   0.,  -4.,   0.],
        [ -2.,  -1.,  79.,   0.,  -4.,   0.],
        ...,
        [ -2.,  -1.,  78.,   0.,  -5.,   0.],
        [ -3.,  -1.,  80.,   1.,  -4.,   1.],
        [ -2.,  -1.,  79.,   0.,  -4.,   0.]],

       [[  6.,  -1.,  79.,  -2.,   4.,  -3.],
        [ 42.,  -3.,  80.,   5.,  53.,   3.],
        [ -5.,   4.,  74., -15., -10.,  -1.],
        ...,
        [ -1.,  -5.,  80.,   6.,  -6.,   0.],
        [ -4.,   5.,  78., -14.,  -9.,  -4.],
        [ -4.,   1.,  80.,  -3., -12.,   5.]],

       [[ -2.,  -6.,  85.,  14.,  -5.,   2.],
        [  0.,   2.,  74.,  -7.,   1.,   0.],
        [ -4.,  -5.,  76.,   7., -11.,   4.],
        ...,
        [  0.,  -9.,  87.,  13.,  -5.,   2.],
        [ -5.,   5.,  67., -17., -16.,   7.],
        [ -6., -10.,  86.,  16., -14.,  -1.]],

       ...,

       [[-12.,  17.,   3., -19., -10.,  -4.],
        [-12.,  12.,  11., -13., -16.,  -4.],
        [ -8.,   3.,  

# Manejo de Datos y Normalización

In [5]:
class_names = ['normal', 'collision_in_tool', 'collision_in_part', 'bottom_collision', 'bottom_obstruction'] 

classes_dict = {'normal':0,
        'collision_in_tool':1,
        'collision_in_part':2,
        'bottom_collision':3,
        'bottom_obstruction':4}
labels_df = labels_df.replace({'labels':classes_dict})
display(labels_df)

def norm(x):
    x_min = x.min()
    x_max = x.max()
    range = x_max - x_min  #min max entre 0 y 1
    return((x-x_min)/(range))

norm_features = norm(features)
input_shape = features.shape
display(norm_features)

,labels
0,0
1,0
2,0
3,0
4,0
...,...
159,1
160,1
161,1
162,1


array([[[0.78027196, 0.7804878 , 0.79818692, 0.78070365, 0.77962443,
         0.78070365],
        [0.78027196, 0.7804878 , 0.79775523, 0.78070365, 0.77984028,
         0.78070365],
        [0.78027196, 0.7804878 , 0.79775523, 0.78070365, 0.77984028,
         0.78070365],
        ...,
        [0.78027196, 0.7804878 , 0.79753939, 0.78070365, 0.77962443,
         0.78070365],
        [0.78005612, 0.7804878 , 0.79797108, 0.78091949, 0.77984028,
         0.78091949],
        [0.78027196, 0.7804878 , 0.79775523, 0.78070365, 0.77984028,
         0.78070365]],

       [[0.7819987 , 0.7804878 , 0.79775523, 0.78027196, 0.78156702,
         0.78005612],
        [0.78976905, 0.78005612, 0.79797108, 0.78178286, 0.79214332,
         0.78135118],
        [0.77962443, 0.78156702, 0.79667602, 0.777466  , 0.77854522,
         0.7804878 ],
        ...,
        [0.7804878 , 0.77962443, 0.79797108, 0.7819987 , 0.77940859,
         0.78070365],
        [0.77984028, 0.78178286, 0.79753939, 0.77768185, 0.778

# Creación del Modelo

In [7]:
def my_model(learning_rate, shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape = shape), # #de neuronas, dimensiones del kernel
        tf.keras.layers.MaxPooling2D((2,2)), # dimensiones del pooling
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(5)
    ])
    
    return model

model = my_model(0.01, input_shape)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 162, 13, 32)       1760      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 81, 6, 32)        0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 15552)             0         
                                                                 
 dense_2 (Dense)             (None, 64)                995392    
                                                                 
 dense_3 (Dense)             (None, 5)                 325       
                                                                 
Total params: 997,477
Trainable params: 997,477
Non-trainable params: 0
________________________________________________